In [1]:
import os
from os.path import join
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import pdb #pdb.pm()

In [3]:
import trading as trd

Using Theano backend.


In [4]:
# relaod module after modifying it
import importlib
importlib.reload(trd.stock_history)
importlib.reload(trd.portfolio)
importlib.reload(trd.benchmarks)
importlib.reload(trd.rl)
importlib.reload(trd.nn)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\code\\python\\cs229\\src\\trading\\__init__.py'>

## The Problem

In [5]:
# percent to have in training data
train_size = 0.8

state_init = {
    # starting cash
    'cash': 1e6, 
    # starting portfolio allocation (%lo, %hi)
    'starting_weights': (0.5, 0.5), 
    # transaction cost to buy/sell a stock
    'trans_cost': 0.001}

base_theta = {
    'non_lin': 'relu',
    'D': 8, 
    'gamma': 0.99,
    'eps': 0.15,
    'H': 100,
    'tau': 0.001}

In [6]:
theta1 = base_theta.copy()
theta1['d'] = 2
theta1['reward'] = trd.sharpe_ratio_reward
theta1['reward_name'] = 'sharpe_reward'

theta2 = base_theta.copy()
theta2['d'] = 7
theta2['reward'] = trd.sharpe_ratio_reward
theta2['reward_name'] = 'sharpe_reward'

theta3 = base_theta.copy()
theta3['d'] = 2
theta3['reward'] = trd.create_penalized_returns_reward(0.5)
theta3['reward_name'] = 'pen_return_l={:s}'.format(str(0.5).replace('.', '_'))

theta4 = base_theta.copy()
theta4['d'] = 7
theta4['reward'] = trd.create_penalized_returns_reward(0.5)
theta4['reward_name'] = 'pen_return_l={:s}'.format(str(0.5).replace('.', '_'))

theta5 = base_theta.copy()
theta5['d'] = 2
theta5['reward'] = trd.create_penalized_returns_reward(0)
theta5['reward_name'] = 'pen_return_l=0'

theta6 = base_theta.copy()
theta6['d'] = 7
theta6['reward'] = trd.create_penalized_returns_reward(0)
theta6['reward_name'] = 'pen_return_l=0'

########################################################################

theta7 = base_theta.copy()
theta7['d'] = 30
theta7['reward'] = trd.sharpe_ratio_reward
theta7['reward_name'] = 'sharpe_reward'

theta8 = base_theta.copy()
theta8['d'] = 30
theta8['reward'] = trd.create_penalized_returns_reward(0)
theta8['reward_name'] = 'pen_return_l=0'

theta9 = base_theta.copy()
theta9['d'] = 30
theta9['reward'] = trd.create_penalized_returns_reward(0.5)
theta9['reward_name'] = 'pen_return_l={:s}'.format(str(0.5).replace('.', '_'))

thetas = [theta7, theta8, theta9]

In [7]:
trd.train_mult_models(thetas, state_init, train_size=train_size, debug=True)

model:      1 of      3
iter:        0	loss:  4.63201e+07     
iter:     5000	loss:  129743          
iter:    10000	loss:  55105.2         
iter:    15000	loss:  3.58872e+06     
iter:    20000	loss:  54110.3         
iter:    25000	loss:  300611          
iter:    30000	loss:  756707          
iter:    35000	loss:  972712          
finished training
model:      2 of      3
iter:        0	loss:  4.81138e+07     
iter:     5000	loss:  6.94808e+07     
iter:    10000	loss:  5.18206e+07     
iter:    15000	loss:  1.5242e+09      
iter:    20000	loss:  1.71979e+08     
iter:    25000	loss:  1.83849e+09     
iter:    30000	loss:  7.15871e+08     
iter:    35000	loss:  3.81763e+09     
finished training
model:      3 of      3
iter:        0	loss:  8.25645e+07     
iter:     5000	loss:  3.10603e+08     
iter:    10000	loss:  9.7139e+08      
iter:    15000	loss:  3.66333e+09     
iter:    20000	loss:  1.1832e+10      
iter:    25000	loss:  4.05351e+10     
iter:    30000	loss:  2.48199e+10 

In [8]:
#pdb.pm()